In [1]:
!pip install gdown

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import sys
sys.path.append('..')
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import os
import torchvision
from torchvision import datasets, models, transforms
from torch.optim.lr_scheduler import StepLR


from src.models import VGG19
from src.dataloader import EuroSatDownloader
from src.training import train_fine_tuning, eval_method, eval_func
from src.dataloader import MiniImageNetDataSet, createEuroSatDataLoaders, EuroSatDownloader
from src.modelvis import visualize_models

plt.ion()

'''
The line cudnn.benchmark = True is typically used in deep learning projects that utilize the CUDA Deep Neural Network (cuDNN) library.
When cudnn.benchmark is set to True, it enables cuDNN to automatically find the best algorithm configuration for the specific 
input sizes and hardware being used. This can result in improved performance during training and inference.By enabling benchmarking, 
cuDNN will run a short benchmarking phase during the first iteration of the model to determine the optimal algorithm configuration. 
This configuration is then cached and used for subsequent iterations, leading to faster execution times.
It's important to note that enabling benchmarking may introduce some overhead during the initial benchmarking phase, so it is typically 
recommended to use it when the input sizes are consistent throughout the training process.
Overall, setting cudnn.benchmark to True can help optimize the performance of deep learning models that use cuDNN.
'''
torch.backends.cudnn.benchmark = True

'''
The line plt.ion() is a function call that activates interactive mode in matplotlib.
When interactive mode is enabled, any plot that is created will be displayed immediately 
and can be updated dynamically. This means that you can modify the plot after it is displayed, 
such as changing the data or adding annotations, and the changes will be reflected in real-time.
'''
plt.ion()

In [3]:
print(torchvision.__version__)
print(torch.__version__)

0.9.2
1.8.2


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


### Download EuroSat Database

In [5]:
extractDir = '..\\data'
url = "https://zenodo.org/records/7711810/files/EuroSAT_RGB.zip?download=1"
downloader = EuroSatDownloader(url, extractDir)
dataDir = downloader.download()

2024-01-07 05:57:05,839 - INFO - EuroSat database already exists.


# Hyperparameters

In [6]:
LEARNING_RATE = 0.01
WEIGHT_DECAY = 0.001
NUM_EPOCHS = 20
BATCH_SIZE = 25
MOMENTUM = 0.09
STEP_SIZE = 5
num_of_classes = 64
EPISODES = 5
GAMMA = 0.5

In [7]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
        transforms.RandomHorizontalFlip(),
        transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
}

# overall
# Overall Mean of RGB channels: tensor([0.3460, 0.3829, 0.4096])
# Overall Standard Deviation of RGB channels: tensor([0.0913, 0.0630, 0.0522])

#pasture
#tensor([0.1518, 0.2503, 0.2958])
#tensor([0.0136, 0.0150, 0.0087])

# highway
#tensor([0.2684, 0.3432, 0.3769])
#tensor([0.0997, 0.0726, 0.0556])

dataDir = '..\\data\\EuroSAT_RGB'
modelPath = '..\\data\\models\\best_model_VGG19_fixedTransformers.pth'

In [8]:
print('Please wait patiently, it may take some seconds...')
total_test_acc = 0.0

good_classes = [set() for _ in range(EPISODES)]
bad_classes = set()

for episode in range(EPISODES):
    print(f"\nTraining Episode: {episode + 1}...")
    
    loaded_model = torch.load(modelPath, map_location=device)
    model = VGG19(64)
    model.load_state_dict(loaded_model)
    model.model.head.fc = nn.Linear(model.model.head.fc.in_features, 5).to(device)

    for param in model.parameters():
        param.requires_grad = True
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(params = model.parameters(), lr = LEARNING_RATE, momentum = MOMENTUM, weight_decay = WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = STEP_SIZE, gamma = GAMMA)
    dataloaders, class_names, dataset_sizes = createEuroSatDataLoaders(data_transforms, dataDir, split=0.25, batch_size = BATCH_SIZE)
    print(dataset_sizes)
    
    model, _, _ = train_fine_tuning(model, dataloaders, criterion, optimizer, scheduler, num_epochs = NUM_EPOCHS, learning_rate = LEARNING_RATE)
    test_loader = dataloaders['test']
    acc_test, test_loss = eval_method(net=model, data_loader=test_loader)
    total_test_acc += acc_test
    if acc_test < 0.6:
        bad_classes.update(class_names)
    elif acc_test > 0.7:
        good_classes[episode].update(class_names)
    print(class_names)
    print(f"Episode {episode + 1}: Test Accuracy: {acc_test:.4f}")

average_test_acc = total_test_acc / EPISODES
print("\nAverage Test Accuracy:", average_test_acc)

Please wait patiently, it may take some seconds...

Training Episode: 1...


2024-01-07 05:57:13,503 - INFO - Loading pretrained weights from url (https://download.pytorch.org/models/vgg19-dcbb9e9d.pth)


{'train': 25, 'test': 75}
epoch: 0, accuracy: 0.040000, avg. loss: 1.956563, test accuracy: 0.653333
epoch: 1, accuracy: 0.840000, avg. loss: 0.406671, test accuracy: 0.853333
epoch: 2, accuracy: 1.000000, avg. loss: 0.119296, test accuracy: 0.906667
epoch: 3, accuracy: 0.960000, avg. loss: 0.070332, test accuracy: 0.893333
epoch: 4, accuracy: 1.000000, avg. loss: 0.026844, test accuracy: 0.906667
epoch: 5, accuracy: 0.960000, avg. loss: 0.059492, test accuracy: 0.906667
epoch: 6, accuracy: 1.000000, avg. loss: 0.020122, test accuracy: 0.906667
epoch: 7, accuracy: 1.000000, avg. loss: 0.016955, test accuracy: 0.906667
epoch: 8, accuracy: 1.000000, avg. loss: 0.020599, test accuracy: 0.906667
epoch: 9, accuracy: 1.000000, avg. loss: 0.021218, test accuracy: 0.906667
epoch: 10, accuracy: 1.000000, avg. loss: 0.015043, test accuracy: 0.906667
epoch: 11, accuracy: 1.000000, avg. loss: 0.015962, test accuracy: 0.906667
epoch: 12, accuracy: 1.000000, avg. loss: 0.011298, test accuracy: 0.906

2024-01-07 06:01:37,050 - INFO - Loading pretrained weights from url (https://download.pytorch.org/models/vgg19-dcbb9e9d.pth)


{'train': 25, 'test': 75}
epoch: 0, accuracy: 0.120000, avg. loss: 1.715028, test accuracy: 0.426667
epoch: 1, accuracy: 0.880000, avg. loss: 0.446824, test accuracy: 0.520000
epoch: 2, accuracy: 1.000000, avg. loss: 0.151307, test accuracy: 0.573333
epoch: 3, accuracy: 0.960000, avg. loss: 0.098891, test accuracy: 0.600000
epoch: 4, accuracy: 0.960000, avg. loss: 0.115564, test accuracy: 0.613333
epoch: 5, accuracy: 0.920000, avg. loss: 0.116700, test accuracy: 0.586667
epoch: 6, accuracy: 1.000000, avg. loss: 0.061740, test accuracy: 0.626667
epoch: 7, accuracy: 1.000000, avg. loss: 0.018169, test accuracy: 0.626667
epoch: 8, accuracy: 1.000000, avg. loss: 0.038902, test accuracy: 0.546667
epoch: 9, accuracy: 1.000000, avg. loss: 0.010448, test accuracy: 0.560000
epoch: 10, accuracy: 1.000000, avg. loss: 0.008144, test accuracy: 0.560000
epoch: 11, accuracy: 1.000000, avg. loss: 0.018366, test accuracy: 0.586667
epoch: 12, accuracy: 1.000000, avg. loss: 0.017523, test accuracy: 0.573

...

{'train': 25, 'test': 75}
epoch: 0, accuracy: 0.040000, avg. loss: 1.855824, test accuracy: 0.653333
epoch: 1, accuracy: 0.880000, avg. loss: 0.527990, test accuracy: 0.773333
epoch: 2, accuracy: 0.920000, avg. loss: 0.270029, test accuracy: 0.733333
epoch: 3, accuracy: 0.960000, avg. loss: 0.176854, test accuracy: 0.906667
epoch: 4, accuracy: 1.000000, avg. loss: 0.046079, test accuracy: 0.880000
epoch: 5, accuracy: 1.000000, avg. loss: 0.035495, test accuracy: 0.893333
epoch: 6, accuracy: 1.000000, avg. loss: 0.037771, test accuracy: 0.893333
epoch: 7, accuracy: 1.000000, avg. loss: 0.030600, test accuracy: 0.893333
epoch: 8, accuracy: 1.000000, avg. loss: 0.021933, test accuracy: 0.893333
epoch: 9, accuracy: 1.000000, avg. loss: 0.014903, test accuracy: 0.893333
epoch: 10, accuracy: 1.000000, avg. loss: 0.013918, test accuracy: 0.893333
epoch: 11, accuracy: 1.000000, avg. loss: 0.012297, test accuracy: 0.893333
epoch: 12, accuracy: 1.000000, avg. loss: 0.014719, test accuracy: 0.893

2024-01-07 06:08:32,674 - INFO - Loading pretrained weights from url (https://download.pytorch.org/models/vgg19-dcbb9e9d.pth)


{'train': 25, 'test': 75}
epoch: 0, accuracy: 0.200000, avg. loss: 1.779152, test accuracy: 0.520000
epoch: 1, accuracy: 0.640000, avg. loss: 0.929458, test accuracy: 0.560000
epoch: 2, accuracy: 0.760000, avg. loss: 0.836164, test accuracy: 0.760000
epoch: 3, accuracy: 0.960000, avg. loss: 0.286932, test accuracy: 0.706667
epoch: 4, accuracy: 1.000000, avg. loss: 0.127851, test accuracy: 0.706667
epoch: 5, accuracy: 1.000000, avg. loss: 0.071696, test accuracy: 0.720000
epoch: 6, accuracy: 1.000000, avg. loss: 0.058193, test accuracy: 0.720000
epoch: 7, accuracy: 1.000000, avg. loss: 0.044010, test accuracy: 0.720000
epoch: 8, accuracy: 1.000000, avg. loss: 0.026502, test accuracy: 0.720000
epoch: 9, accuracy: 1.000000, avg. loss: 0.030723, test accuracy: 0.706667
epoch: 10, accuracy: 1.000000, avg. loss: 0.028890, test accuracy: 0.733333
epoch: 11, accuracy: 1.000000, avg. loss: 0.033680, test accuracy: 0.733333
epoch: 12, accuracy: 1.000000, avg. loss: 0.024792, test accuracy: 0.733

2024-01-07 06:11:21,963 - INFO - Loading pretrained weights from url (https://download.pytorch.org/models/vgg19-dcbb9e9d.pth)


{'train': 25, 'test': 75}
epoch: 0, accuracy: 0.280000, avg. loss: 1.706629, test accuracy: 0.720000
epoch: 1, accuracy: 0.800000, avg. loss: 0.491093, test accuracy: 0.626667
epoch: 2, accuracy: 0.680000, avg. loss: 0.775065, test accuracy: 0.680000
epoch: 3, accuracy: 0.680000, avg. loss: 1.153667, test accuracy: 0.733333
epoch: 4, accuracy: 0.920000, avg. loss: 0.356677, test accuracy: 0.826667
epoch: 5, accuracy: 1.000000, avg. loss: 0.144526, test accuracy: 0.853333
epoch: 6, accuracy: 1.000000, avg. loss: 0.080384, test accuracy: 0.853333
epoch: 7, accuracy: 1.000000, avg. loss: 0.064765, test accuracy: 0.840000
epoch: 8, accuracy: 1.000000, avg. loss: 0.075202, test accuracy: 0.826667
epoch: 9, accuracy: 1.000000, avg. loss: 0.045965, test accuracy: 0.826667
epoch: 10, accuracy: 1.000000, avg. loss: 0.037618, test accuracy: 0.826667
epoch: 11, accuracy: 1.000000, avg. loss: 0.033627, test accuracy: 0.826667
epoch: 12, accuracy: 1.000000, avg. loss: 0.026389, test accuracy: 0.826

In [9]:
for good_set in good_classes:
    print(bad_classes - good_set)
print(bad_classes)

{'River', 'Industrial', 'Pasture', 'Highway'}
{'River', 'Pasture', 'Industrial', 'Highway', 'HerbaceousVegetation'}
{'Highway', 'HerbaceousVegetation'}
{'Pasture', 'HerbaceousVegetation'}
{'River'}
{'River', 'Pasture', 'Industrial', 'Highway', 'HerbaceousVegetation'}
